In [2]:
# import commands
import nltk
from nltk.tokenize import word_tokenize
# from nltk import tokenize
from nltk.corpus import stopwords
import string
import textmining
import os
# import csv
# import string
import numpy as np
# import textmining
import math
# import pandas as pd
from numpy import dot
from numpy.linalg import norm
# from nltk.stem import WordNetLemmatizer
nltk.download('wordnet') 

[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
def strip_punctuation(s):
    table = str.maketrans({key: ' ' for key in string.punctuation})
    return s.translate(table)

def tokenizeSent(text, delim="\n"):
    sentences = [t for t in text.split(delim) if len(t) > 0]
    return sentences

def filterSent(sentences):
        # file = open(inputFile, 'r',encoding='Latin-1')
        # text = file.read()      

        # text = text.replace('\n',' ')
        
        # words_count = len(word_tokenize(strip_punctuation(text)))

        # sentences = text.split("\n")
        # Tokenize the text into sentences
        # sentences = sent_tokenize(text)
        #Original Sentences
        sentences_backup = list(sentences)
        
        filtered_sentences = []
        
        stop_words = set(stopwords.words('english'))
        word_list = []
        for i in range(len(sentences_backup)):
            temp = []
            
            word_tokens = word_tokenize(strip_punctuation(sentences_backup[i]))
            # Initialize the WordNet lemmatizer
            # lemmatizer = WordNetLemmatizer()

            # # Lemmatize each word
            # lemmatized_words = [lemmatizer.lemmatize(word) for word in word_tokens]

            for w in word_tokens:
            # for w in lemmatized_words:    
                if len(w)>2 and w.lower() not in stop_words:
                    word_lower = w.lower()
                    temp.append(word_lower)
                    word_list.append(word_lower)
            filtered_sentences.append(temp)

        for i in range(len(filtered_sentences)):
                filtered_sentences[i]=" ".join(filtered_sentences[i])
                
        return filtered_sentences, list(set(word_list))


def fullTermSentMatrix(sentencesList):
        tdm = textmining.TermDocumentMatrix()

        for sent in sentencesList:
                tdm.add_doc(sent)

         
        temp = list(tdm.rows(cutoff=1))
        vocab = tuple(temp[0])
        X = np.array(temp[1:],dtype = 'float64')
        termSentenceMatrix = X.transpose()
        
        # Apply tf-idf for matrix normalization
        
        # Initialize the TF-IDF transformer
        # tfidf_transformer = TfidfTransformer()
        
        # # Fit and transform the matrix to calculate TF-IDF values
        # tfidf_matrix = tfidf_transformer.fit_transform(termSentenceMatrix)
        # # The result is a sparse matrix, you can convert it to an array
        # termSentence_tfidf_array = tfidf_matrix.toarray()
        return (termSentenceMatrix , vocab)

def findTermVectorSVD(termSentMatrix):
        U_doc,E1,V_doc= np.linalg.svd(termSentMatrix, full_matrices=0, compute_uv=1)
        np.set_printoptions(suppress=True)

##        E_doc = np.diag(E1,k=0)
        docTermVector = np.zeros(termSentMatrix.shape[0],dtype ='float64')
        for i in range(U_doc.shape[0]):
                temp = 0.0
                ## 2012 method
                for j in range(U_doc.shape[1]):
                        temp += (U_doc[i][j] * E1[j] *E1[j])*(U_doc[i][j] * E1[j] *E1[j])
                docTermVector[i] =  math.sqrt(temp)
##        print(docTermVector)
        return docTermVector

def findTermVector(corpusTermsDict,wordList):
    termVector = np.zeros(len(corpusTermsDict), dtype=np.float64)
    # Enumerate the dictionary
    for index, (word, value) in enumerate(corpusTermsDict.items()):
        # print(f' {index}:{word}:{value}')
        if word in wordList:
            # print(f'word:value= {word}:{value}')
            termVector[index] = value
    
    return termVector

def findSimilarity(termVector1, termVector2):
        sim = dot(termVector1, termVector2)/(norm(termVector1)*norm(termVector2))
        return sim

In [30]:
no_of_documents = 50
k=2
similarityMatrix = np.zeros((no_of_documents,k),dtype='float64')
for i in range(no_of_documents):
        for j in range(k):
                similarityMatrix[i][j] = -1 

i=0
# algo = 'citation_InLegalBert'
# algo = 'CaseSummarizer'
# algo = 'MMR'
# algo = 'Legal-pegasus'
algo = 'Legal-LED'
# cbs_algo = ''
# cbs_algo = 'baseline-top5-summingScores'
# cbs_algo = 'Additive-top5-summing-scores-normalized'
# cbs_algo ='V3-onlyCosineColumnSum'
# cbs_algo = 'V3-onlyCosineColumnSum-Lnormalized'
# cbs_algo = 'V1-top5-normalizedLength-sorted-row-column'
cbs_algo = ''
# cbs_algo = 'NMF-Normalized-TP-binary'
# cbs_algo = 'NMF-Normalized-TR-binary'
# cbs_algo = 'NMF-Normalized-ciSumm'
# cbs_algo = 'NMF-Normalized-additive'
# cbs_algo= 'NMF-Normalized-citationDiversity'

In [31]:
import pandas as pd

doc_df = pd.read_csv('../results/case-similarity-summaries-llama2-70b-chat-030324-final.csv')
doc_df.head()

,jid,prompt,summary
0,91635,[INST]You are given a part of a court judgment...,The court judgment is about a dispute over a...
1,374340,[INST]You are given a part of a court judgment...,The court judgment is regarding an appeal ag...
2,1132299,[INST]You are given a part of a court judgment...,The court judgment is regarding a trade mark...
3,70487596,[INST]You are given a part of a court judgment...,This court judgment is regarding an applicat...
4,60799,[INST]You are given a part of a court judgment...,The Supreme Court of India allowed several c...


In [45]:
for doc_id, llama2_sum in zip(doc_df['jid'], doc_df['summary']):
    with open(f'../results/case-similarity-text/tail/{doc_id}.txt') as f:
        s1 = f.read()
    refSummSentences = tokenizeSent(s1)
    sysSummSentences = [t.strip() + '.' for t in tokenizeSent(llama2_sum, delim='. ')]

    # Combine the sentence lists into a single list.
    # combined_sentences = docSentences + refSummSentences + sysSummSentences + MMRSentences
    combined_sentences = refSummSentences + sysSummSentences 
    # combined_sentences = refSummSentences + MMRSentences 
    
    # Create a list of unique sentences by converting the list to a set and back to a list.
    unique_sentences = list(set(combined_sentences))

    corpus_sentences, _ = filterSent(unique_sentences) # remove stop words and words of length less than 2
    (termSentenceMatrix , vocab)= fullTermSentMatrix(corpus_sentences)
    termVectorCorpus = findTermVectorSVD(termSentenceMatrix)

    corpusTermsDict = dict(zip(vocab, termVectorCorpus))

    _ , refSummWords = filterSent(refSummSentences)
    _ , summ_1_Words = filterSent(sysSummSentences)

    # termVectorDoc = findTermVector(corpusTermsDict,docWords)
    termVectorRefSumm = findTermVector(corpusTermsDict,refSummWords)
    termVectorSysSumm1 = findTermVector(corpusTermsDict,summ_1_Words)
    # termVectorSysSumm2 = findTermVector(corpusTermsDict, summ_2_Words) 
    
    # ref_id_name = os.path.splitext(ref_id)[0]
    similarityMatrix[i][0]= doc_id
    similarityMatrix[i][1]= findSimilarity(termVectorRefSumm,termVectorSysSumm1)

    i+=1

In [46]:
similarityMatrix

array([[9.16350000e+04, 8.26299317e-01],
       [3.74340000e+05, 9.08293098e-01],
       [1.13229900e+06, 9.21415443e-01],
       [7.04875960e+07, 8.04095029e-01],
       [6.07990000e+04, 8.04021486e-01],
       [3.80655000e+05, 9.27222478e-01],
       [1.21302100e+06, 8.99552957e-01],
       [6.23494000e+05, 8.66084716e-01],
       [1.33466500e+06, 6.95676835e-01],
       [6.11025000e+05, 8.20387026e-01],
       [7.95465000e+05, 9.18535095e-01],
       [3.12398950e+07, 9.41054928e-01],
       [1.39299795e+08, 9.03305206e-01],
       [2.48715000e+05, 9.30330485e-01],
       [1.23967300e+06, 9.23334482e-01],
       [1.13963352e+08, 9.07570628e-01],
       [1.87854800e+06, 9.42375404e-01],
       [3.14044000e+05, 8.69927181e-01],
       [1.38192511e+08, 6.89055550e-01],
       [1.48981860e+07, 9.45512762e-01],
       [1.16550300e+06, 9.22630982e-01],
       [8.91908900e+06, 8.44122778e-01],
       [1.10813550e+08, 7.79966895e-01],
       [4.35400000e+03, 8.09196674e-01],
       [5.486531

In [50]:
sim_df = pd.DataFrame(similarityMatrix, columns=['doc_id', 'simscore'])
sim_df

,doc_id,simscore
0,91635.0,0.826299
1,374340.0,0.908293
2,1132299.0,0.921415
3,70487596.0,0.804095
4,60799.0,0.804021
5,380655.0,0.927222
6,1213021.0,0.899553
7,623494.0,0.866085
8,1334665.0,0.695677
9,611025.0,0.820387


In [51]:
sim_df.to_csv('../results/summarization-similarity-scores.csv')

In [52]:
sim_df['simscore'].argmax()

45

In [54]:
sim_df.iloc[45,:]['doc_id']

1462242.0

In [56]:
doc_df[doc_df['jid'] == 1462242]['summary'][45]

"  The summary of this court judgment is as follows:\n\nThe appellants (plaintiffs) had filed a case in the civil court, which was later appealed to the first appellate court and then to the High Court. The High Court had decided the case only on the ground of jurisdiction and not on other issues involved. The appellants' counsel argued that since the trial court and first appellate court had decided all issues in favor of the appellants, the judgment of the first appellate court should be confirmed. However, the Supreme Court did not accept this contention and held that it would not be fair to the respondents (defendants) to accept the finding of the first appellate court without the same being examined by the High Court. Therefore, the Supreme Court allowed the appeal, set aside the judgment of the High Court, and remanded the case to the High Court for deciding the other issues involved on merits. The decision was made without any prejudice to the respondents' rights, if any, under 

In [57]:
doc_df[doc_df['jid'] == 1462242]['prompt'][45]

"[INST]You are given a part of a court judgment along with which case it belongs to. You are a legal expert.\n\n\n### Judgment Excerpt: that the civil court had no jurisdiction.\n8\\. The learned counsel for the appellants, Shri R.B. Datar, however,\ncontended that the trial court as well as the first appellate court had\ndecided all the issues involved in the case in favour of the plaintiffs-\nappellants and in case we come to the conclusion, as indeed we have come to,\nthat the jurisdiction of the civil court is not barred, the judgment of the\nfirst appellate court should be confirmed and there is no need to remand the\ncase again to the High Court for deciding it afresh on merits.\n9\\. We are not inclined to accept the contention raised by the counsel for the\nappellants. A number of issues were involved in the case and the High Court\nhas decided the case only on the ground of jurisdiction and no other point has\nbeen dealt with. It will not be fair to the respondents to accept t

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer
from rouge import Rouge
from rouge_score import rouge_scorer

In [65]:
from tqdm.notebook import tqdm

bert_scores = []
rouge_scores = []
zipped = zip(doc_df['jid'], doc_df['summary'])
for doc_id, llama2_sum in tqdm(zipped):
    with open(f'../results/case-similarity-text/tail/{doc_id}.txt') as f:
        s1 = f.read()
    bert_scorer = BERTScorer(model_type='bert-base-uncased')
    bert_score = bert_scorer.score([s1], [llama2_sum])
    bert_scores.append(bert_score)
    _rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_score = _rouge_scorer.score(s1, llama2_sum)
    rouge_scores.append(rouge_score)

0it [00:00, ?it/s]

In [96]:
r1_scores = [s['rouge1'] for s in rouge_scores]
r2_scores = [s['rouge2'] for s in rouge_scores]
rl_scores = [s['rougeL'] for s in rouge_scores]

In [75]:
bert_precs = [score[0][0].detach().numpy() for score in bert_scores]
bert_recs = [score[1][0].detach().numpy() for score in bert_scores]
bert_f1s = [score[2][0].detach().numpy() for score in bert_scores]

In [78]:
bert_sim_df = pd.DataFrame({
    'doc_id': doc_df['jid'],
    'bertscore_p': bert_precs,
    'bertscore_r': bert_recs,
    'bertscore_f1': bert_f1s,
})
bert_sim_df

,doc_id,bertscore_p,bertscore_r,bertscore_f1
0,91635,0.534994,0.6734154,0.59627676
1,374340,0.635574,0.8443355,0.7252304
2,1132299,0.63420516,0.80951715,0.7112171
3,70487596,0.4831831,0.6919023,0.5690063
4,60799,0.4886328,0.6609679,0.5618831
5,380655,0.6417969,0.79507285,0.7102597
6,1213021,0.44832233,0.7492764,0.5609848
7,623494,0.6412593,0.8010489,0.71230274
8,1334665,0.4917019,0.7120296,0.58170164
9,611025,0.5949189,0.7617523,0.66807765


In [79]:
bert_sim_df.to_csv('../results/summarization-similarity-scores-bertscore.csv', index=False)

In [80]:
bert_sim_df['bertscore_f1'].argmax()

10

In [81]:
bert_sim_df['bertscore_p'].argmax()

10

In [82]:
bert_sim_df['bertscore_r'].argmax()

1

In [86]:
bert_sim_df.iloc[10,:]['doc_id']

795465

In [88]:
doc_df[doc_df['jid'] == 795465]['summary'][10]

'  The Supreme Court of India has delivered a judgment in a batch of civil appeals, writ petitions, and a transferred case, all related to the issue of taxation of inter-State sales. The Court has held that the decision of the Bombay High Court in 20th Century Finance Corporation Ltd. vs. State of Maharashtra is erroneous, while the decision of the Andhra Pradesh High Court in ITC Classics Finance & Services vs. Commissioner of Commercial Taxes is affirmed.\n\nThe Court has allowed all the civil appeals, writ petitions, and the transferred case, except for civil appeal Nos. 6218-23/95, and has set aside the judgments under appeal, except for CA Nos. 6218-23/95. The Court has also directed the respondents to assess the appellants and petitioners for the relevant years involved in these cases only in accordance with the principles of law laid down in this judgment.\n\nIn summary, the Supreme Court has ruled that inter-State sales are not liable to taxation under the Sales Tax Act, and ha

In [90]:
doc_df[doc_df['jid'] == 795465]['prompt'][10]

'[INST]You are given a part of a court judgment along with which case it belongs to. You are a legal expert.\n\n\n### Judgment Excerpt: India and (iii) an inter-State sale.\n(53) Following what we have stated above, we are of the opinion that the\ndecision of the Bombay High Court in 20th Century Finance Corporation Ltd. Vs.\nState of Maharashtra (1989) 75 STC, 217 (under appeal) is erroneous, whereas,\nwe affirm the decision of the Andhra Pradesh High Court in ITC Classics\nFinance & Services vs. Commissioner of Commercial Taxes (1995) 97 STC, 330\n(under appeal).\n(54) For the aforesaid reasons, all the civil appeals, writ petitions and the\ntransferred case, excepting civil appeal Nos. 6218-23/95, deserve to be\nallowed and civil appeal Nos. 6218- 23/95 deserve to be dismissed.\nConsequently, we set aside the judgments under appeal excepting in CA\nNos.6218-23/95. All the civil appeals excepting C.A. Nos.6218-23/95, writ\npetitions under Article 32, the transferred case and the writ

In [100]:
r1_precs = [score.precision for score in r1_scores]
r1_recs = [score.recall for score in r1_scores]
r1_f1s = [score.fmeasure for score in r1_scores]

In [102]:
r2_precs = [score.precision for score in r2_scores]
r2_recs = [score.recall for score in r2_scores]
r2_f1s = [score.fmeasure for score in r2_scores]

In [103]:
rl_precs = [score.precision for score in rl_scores]
rl_recs = [score.recall for score in rl_scores]
rl_f1s = [score.fmeasure for score in rl_scores]

In [105]:
rouge_sim_df = pd.DataFrame({
    'doc_id': doc_df['jid'],
    'rouge1_p': r1_precs,
    'rouge1_r': r1_recs,
    'rouge1_f1': r1_f1s,
    'rouge2_p': r2_precs,
    'rouge2_r': r2_recs,
    'rouge2_f1': r2_f1s,
    'rougeL_p': rl_precs,
    'rougeL_r': rl_recs,
    'rougeL_f1': rl_f1s,
    
})
rouge_sim_df

,doc_id,rouge1_p,rouge1_r,rouge1_f1,rouge2_p,rouge2_r,rouge2_f1,rougeL_p,rougeL_r,rougeL_f1
0,91635,0.735294,0.298805,0.424929,0.277228,0.112000,0.159544,0.382353,0.155378,0.220963
1,374340,0.929293,0.347170,0.505495,0.653061,0.242424,0.353591,0.777778,0.290566,0.423077
2,1132299,0.892857,0.401606,0.554017,0.585586,0.262097,0.362117,0.508929,0.228916,0.315789
3,70487596,0.778947,0.289062,0.421652,0.382979,0.141176,0.206304,0.536842,0.199219,0.290598
4,60799,0.706522,0.303738,0.424837,0.318681,0.136150,0.190789,0.489130,0.210280,0.294118
5,380655,0.857143,0.430279,0.572944,0.624000,0.312000,0.416000,0.626984,0.314741,0.419098
6,1213021,0.786260,0.565934,0.658147,0.538462,0.386740,0.450161,0.374046,0.269231,0.313099
7,623494,0.854839,0.439834,0.580822,0.609756,0.312500,0.413223,0.758065,0.390041,0.515068
8,1334665,0.598425,0.374384,0.460606,0.349206,0.217822,0.268293,0.370079,0.231527,0.284848
9,611025,0.803279,0.379845,0.515789,0.512397,0.241245,0.328042,0.598361,0.282946,0.384211


In [106]:
rouge_sim_df.to_csv('../results/summarization-similarity-scores-rougescores.csv', index=False)

In [107]:
rouge_sim_df['rouge1_f1'].argmax() # 1462242

45

In [108]:
rouge_sim_df['rouge2_f1'].argmax() # 1543623

48

In [112]:
rouge_sim_df['rougeL_f1'].argmax() # 1543623

48

In [113]:
doc_df[doc_df['jid'] == 1462242]['prompt'][45]

"[INST]You are given a part of a court judgment along with which case it belongs to. You are a legal expert.\n\n\n### Judgment Excerpt: that the civil court had no jurisdiction.\n8\\. The learned counsel for the appellants, Shri R.B. Datar, however,\ncontended that the trial court as well as the first appellate court had\ndecided all the issues involved in the case in favour of the plaintiffs-\nappellants and in case we come to the conclusion, as indeed we have come to,\nthat the jurisdiction of the civil court is not barred, the judgment of the\nfirst appellate court should be confirmed and there is no need to remand the\ncase again to the High Court for deciding it afresh on merits.\n9\\. We are not inclined to accept the contention raised by the counsel for the\nappellants. A number of issues were involved in the case and the High Court\nhas decided the case only on the ground of jurisdiction and no other point has\nbeen dealt with. It will not be fair to the respondents to accept t

In [115]:
doc_df[doc_df['jid'] == 1462242]['summary'][45]

"  The summary of this court judgment is as follows:\n\nThe appellants (plaintiffs) had filed a case in the civil court, which was later appealed to the first appellate court and then to the High Court. The High Court had decided the case only on the ground of jurisdiction and not on other issues involved. The appellants' counsel argued that since the trial court and first appellate court had decided all issues in favor of the appellants, the judgment of the first appellate court should be confirmed. However, the Supreme Court did not accept this contention and held that it would not be fair to the respondents (defendants) to accept the finding of the first appellate court without the same being examined by the High Court. Therefore, the Supreme Court allowed the appeal, set aside the judgment of the High Court, and remanded the case to the High Court for deciding the other issues involved on merits. The decision was made without any prejudice to the respondents' rights, if any, under 

In [114]:
doc_df[doc_df['jid'] == 1543623]['prompt'][48]

'[INST]You are given a part of a court judgment along with which case it belongs to. You are a legal expert.\n\n\n### Judgment Excerpt: lakes. After the Report is received, the matter may be listed. Point 6 is\ndecided accordingly. In the result, the appeal is allowed, the judgment of the\nHigh Court and the order of the appellate authority under section 28 of the\nWater Act, 1974 are set aside and the order of the appellant Board refusing\npermission to the seventh respondent under section 25 of the Water Act is\nrestored. Before parting with the case, we acknowledge the excellent Reports\nsubmitted to this Court by the three expert bodies on the basis of\nscientific/technological research of a very high order. The amount of hard\nwork done by these three bodies is commendable. But for these expert reports\nit would have been very difficult for this Court to resolve the complicated\nscientific issues involved in this case, with confidence. It will be open to\nthe three expert bodies (

In [116]:
doc_df[doc_df['jid'] == 1543623]['summary'][48]

'  The court judgment is in regards to an appeal against a judgment of the High Court and an order of an appellate authority under the Water Act, 1974. The appeal was allowed and the judgment and order were set aside. The order of the appellant Board refusing permission under section 25 of the Water Act was restored. The court acknowledged the excellent reports submitted by three expert bodies and their hard work in resolving the complicated scientific issues involved in the case. The court directed the three expert bodies to submit their list of expenses or fee, if any, to the State of Andhra Pradesh through the appellant Board, and if any claims for monies are made, the same shall be paid by the State of Andhra Pradesh. The matter was listed after 4 months, after the Report of the State of Andhra Pradesh as directed above, is received.'